<a href="https://colab.research.google.com/github/vizzies/grcml-challenge-a-starter/blob/main/starter-notebooks/grcml_challenge_a_shruti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Uncomment the below lines if these dependencies are not already installed
!pip install --upgrade scispacy
!pip install --upgrade spacy
!pip install --upgrade gensim
!pip install --upgrade pyLDAvis

# Uncomment the below line if en_core_sci_sm is not loaded on your system.
!pip install --upgrade https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

#uncomment if you need to read the data files from google cloud storage
#from google.cloud import storage

#uncomment if you need to read data in from Google Drive that's mounted to a CoLab notebook
from google.colab import drive

import time
import bokeh
from sklearn.manifold import TSNE
import pandas as pd
import scispacy
import spacy
import string
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim import similarities
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords  
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial.distance import jensenshannon
import numpy as np
from google.cloud import storage

#nlp = spacy.load('en_core_sci_sm')
import en_core_sci_sm
nlp = en_core_sci_sm.load()

import nltk
# Uncomment the below line if nltk's stopwords are not already on your system.
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer('english')

from bokeh.models import HoverTool, CustomJS, ColumnDataSource
from bokeh.palettes import viridis, Viridis256, magma, Turbo256, linear_palette
from bokeh.transform import factor_cmap
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 10.0MB 10.8MB/s 
     |████████████████████████████████| 13.0MB 38.8MB/s 
     |████████████████████████████████| 2.1MB 36.8MB/s 
     |████████████████████████████████| 296kB 42.4MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.3.2)
     |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 1.6MB 2.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=59d55be6838453dd022539cf15b92cac8071cc16420f9054ee7eb351c3c15823
  Stored in 

/usr/local/lib/python3.6/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.6/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_sci_sm' (0.2.4) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
import pandas as pd
# df = pd.read_csv("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", sep=" ", header=None)
# df.columns = ["a", "b", "c", "etc."]
# df
data = open("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", "r")
print(data.read())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
with open("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", "r") as infile:
    data = infile.readlines()
    
print("\n\n".join((data[0][:500], data[1][:500], data[2][:500])))

doi:10.1016/j.ijheatmasstransfer.2007.11.028 Available online at www.sciencedirect.com www.elsevier.com/locate/ijhmt  International Journal of Heat and Mass Transfer 51 (2008) 33253341 Analysis of the 1D heat conduction problem for a single fin with temperature dependent heat transfer coefficient  Part II.  Optimum characteristics of straight plate and cylindrical pin fins  I.N. Dulkin *, G.I. Garasko  All-Russian Electrical Engineering Institute, 12 Krasnokazarmennaya Street, 111250 Moscow, Rus

Heat transfer and flow structure characterization for pin fins produced by cold spray additive manufacturing International Journal of Heat and Mass Transfer 98 (2016) 650661 Contents lists available at ScienceDirect  International Journal of Heat and Mass Transfer  journal homepage: www.elsevier .com/locate / i jhmt Heat transfer and flow structure characterization for pin fins produced by cold spray additive manufacturing http://dx.doi.org/10.1016/j.ijheatmasstransfer.2016.03.069 0017-9310/ 2

In [31]:
def get_entities(temp_article):
    '''
    A function for preprocessing an article's text.  Spacy's en_core_sci_sm library is used to identify scientfic domain terms. 
    Stopword removal and snowball stemming is then used to cleanup each of term.
    Arg:
        temp_article: the string input for preprocessing
    Returns:
        temp_entities: a list of preprocessed terms
    '''
    if len(temp_article) > nlp.max_length:
        nlp.max_length = len(temp_article)
    doc = nlp(temp_article)
    temp_entities = []
    for n in doc.ents:
        temp_str = ''
        for w in n:
            if w not in stop_words:
                if str(w).isalpha():
                    temp_str += stemmer.stem(str(w)) + ' '
                else:
                    temp_str += str(w) + ' '
        temp_entities.append(temp_str.strip())
    return temp_entities

In [32]:
bow_vector = CountVectorizer(tokenizer=get_entities, min_df=2, max_df=5, ngram_range=(1,3))
data_veccluster_count = 10

In [33]:
cluster_count = 10

# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=cluster_count,               # Number of topics
                                      #doc_topic_prior=1.0,
                                      max_iter=100,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [34]:
lda_output = lda_model.fit_transform(data_vectorized)

In [35]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, bow_vector, mds='tsne', sort_topics=False)
panel

NotFittedError: ignored

In [23]:
top_cluster = []
for n in range(lda_output.shape[0]):
    categ = np.argwhere(lda_output[n,:] == np.amax(lda_output[n,:]))
    top_cluster.append(str(categ[0][0]+1))

In [24]:
test = data_vectorized.todense()

In [25]:
tsne = TSNE(random_state=2017, perplexity=30, n_iter=5000)#, early_exaggeration=120)
embedding = tsne.fit_transform(test)

In [26]:
unique_cats = list(set(top_cluster))
unique_cats_count = len(unique_cats)
print('There are ' + str(unique_cats_count) + ' unique categories in our dataset.')

There are 10 unique categories in our dataset.


In [27]:
viridis_spec = viridis(unique_cats_count)
magma_spec = magma(unique_cats_count)
turbo_spec = linear_palette(Turbo256, unique_cats_count)

In [28]:
source = ColumnDataSource(data=dict(
    x=embedding[:,0],
    y=embedding[:,1],
    top_cluster=top_cluster
))

TOOLTIPS = [
    ("index", '$index'),
    ("Title", '@desc'),
    ('Category', '@top_cluster')
]

In [29]:
p = figure(title="t-SNE Projection of Vectorized Heat Exchangers", tooltips=TOOLTIPS, plot_height=800, plot_width=800)
p.circle('x', 'y', size=10, source=source, fill_alpha=0.75, fill_color=factor_cmap('top_cluster', palette=magma_spec, factors=top_cluster))
show(p)

/usr/local/lib/python3.6/dist-packages/bokeh/models/mappers.py:147: UserWarning: Palette length does not match number of factors. ['7', '9', '6', '7', '4', '9', '5', '7', '6', '2', '5', '10', '9', '4', '1', '10', '7', '6', '2', '6', '4', '10', '6', '7', '6', '6', '10', '7', '3', '10', '5', '7', '9', '2', '10', '4', '7', '3', '2', '3', '2', '2', '9', '4', '4', '10', '4', '5', '7', '4', '5', '8', '10', '6', '7', '4', '7', '9', '8', '3', '9', '2', '9', '9', '1', '10', '3', '5', '4', '8', '8', '3', '5', '10', '8', '5', '3', '5', '7', '9', '2', '2', '8', '1', '6', '8', '6', '7', '10', '4', '1', '10', '6', '1', '6', '9', '9', '3', '7', '1'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))
